In [1]:
import os, json, sys

In [2]:
people_map = {}
day_map = {}
day_group_map = {}
people_to_assign = []
groups = []

In [3]:
dir_path = '/Users/conniexu/Documents/i3Hackathon/sample_student_responses'
for file in os.listdir(dir_path):
    file_path = f"{dir_path}/{file}"
    with open(file_path) as json_file:
        data = json.load(json_file)
        people_to_assign.append(data['full_name'])
        people_map[data['full_name']] = data

In [4]:
people_map

{'Connie Xu': {'full_name': 'Connie Xu',
  'available_day': ['4-4-2021', '4-5-2021', '4-6-2021', '4-7-2021'],
  'group_size': 4,
  'comfort_with_material': 5,
  'focus_scale': 9,
  'comfort_with_helping': 8},
 'Sophie Li': {'full_name': 'Sophie Li',
  'available_day': ['4-6-2021', '4-7-2021', '4-8-2021'],
  'group_size': 4,
  'comfort_with_material': 5,
  'focus_scale': 1,
  'comfort_with_helping': 10},
 'Allen Park': {'full_name': 'Allen Park',
  'available_day': ['4-6-2021', '4-7-2021'],
  'group_size': 3,
  'comfort_with_material': 5,
  'focus_scale': 2,
  'comfort_with_helping': 3},
 'Sachiko Datta': {'full_name': 'Sachiko Datta',
  'available_day': ['4-5-2021', '4-7-2021'],
  'group_size': 7,
  'comfort_with_material': 7,
  'focus_scale': 9,
  'comfort_with_helping': 8},
 'Chris Eisgruber': {'full_name': 'Chris Eisgruber',
  'available_day': ['4-5-2021', '4-6-2021', '4-7-2021'],
  'group_size': 4,
  'comfort_with_material': 5,
  'focus_scale': 2,
  'comfort_with_helping': 3},
 'Pe

In [5]:
for person, info in people_map.items():
    for day in info['available_day']: 
        if day in day_map and person not in day_map[day]:
            day_map[day].append(person)
        else:
            day_map[day] = [person]

In [6]:
for day, people in day_map.items():
    for person in people:
        group_size = people_map[person]['group_size']
        if day in day_group_map:
            if group_size in day_group_map[day] and person not in day_group_map[day][group_size]:
                day_group_map[day][group_size].append(person)
            else: 
                day_group_map[day][group_size] = [person]
        else:
            day_group_map[day] = {group_size: [person]}

In [7]:
day_group_map

{'4-4-2021': {4: ['Connie Xu'], 3: ['Dara Oseyemi']},
 '4-5-2021': {4: ['Connie Xu', 'Chris Eisgruber'],
  7: ['Sachiko Datta'],
  3: ['Peter Bolgucki']},
 '4-6-2021': {4: ['Connie Xu', 'Sophie Li', 'Chris Eisgruber'],
  3: ['Allen Park', 'Dara Oseyemi']},
 '4-7-2021': {4: ['Connie Xu', 'Sophie Li', 'Chris Eisgruber'],
  3: ['Allen Park'],
  7: ['Sachiko Datta']},
 '4-8-2021': {4: ['Sophie Li'], 3: ['Dara Oseyemi']}}

In [8]:
def analyze_characteristics(people_map, people):
    print('analyze chars')

In [9]:
# random prime numbers for hashing
def combine_adjacent(people_map, group_data, date=False):
    if not len(group_data) <= 1:
        min_diff = sys.maxsize
        values_to_combine = []
        date_hash = {}

        group_data_keys = group_data.keys()

        if date:
            hashed_group_data = []
            for day in group_data:
                day_data = day.split('-')
                hash_code = day_data[0] * 31 + day_data[1] * 13 + day_data[2] * 733
                date_hash[hash_code] = day
            group_data_keys = hashed_group_data

        # combine closest groups
        sorted_group_data = sorted(group_data_keys)
        for i in range(len(sorted_group_data) - 1):
            diff = sorted_group_data[i + 1] - sorted_group_data[i]
            if diff < min_diff and not group_data[sorted_group_data[i + 1]] == group_data[sorted_group_data[i]]:
                min_diff = diff
                values_to_combine = [sorted_group_data[i], sorted_group_data[i + 1]]
        
        if not len(values_to_combine) == 0:
            if date:
                group_data[date_hash[values_to_combine[0]]] = group_data[date_hash[values_to_combine[0]]] + group_data[date_hash[values_to_combine[1]]]
            else:
                group_data[values_to_combine[0]] = list(set(group_data[values_to_combine[0]] + group_data[values_to_combine[1]]))
                group_data[values_to_combine[1]] = list(set(group_data[values_to_combine[0]] + group_data[values_to_combine[1]]))


In [10]:
def update_day_group_map(day_group_map, people_to_assign):
    print(day_group_map)
    for day, group_data in day_group_map.items():
        for group in group_data.values():
            i = 0
            while i < len(group):
                person = group[i]
                if person not in people_to_assign:
                    group.remove(person)
                else:
                    i += 1

In [11]:
def assign_remainder(people_to_assign):
    if len(people_to_assign) <= 2 and len(people_to_assign) > 0:
        people_to_add = []
        for i in range(len(people_to_assign)):
            people_to_add.append(people_to_assign[i])
        groups.append(people_to_add)
        people_to_assign = []
        return people_to_assign

In [12]:
tolerance = 0
# check whether groups can be filled
modifications = 1
while modifications > 0:
    modifications = 0
    for day, group_data in day_group_map.items():
        sorted_group_data = sorted(group_data.keys(), reverse=True)
        combined = False
        for i in range(0, len(sorted_group_data)):
            group_size = sorted_group_data[i]
            people = group_data[group_size]
            num_people = 0
            for person in people:
                if person in people_to_assign:
                    num_people += 1
            if abs(num_people - group_size) <= tolerance and num_people > 1:
                modifications += 1
                people_to_add = []
                for i in range(len(people)):
                    people_to_add.append(people[i])
                groups.append(people_to_add)
                for person in people:
                    if person in people_to_assign:
                        people_to_assign.remove(person)
                update_day_group_map(day_group_map, people_to_assign)
        for i in range(0, len(sorted_group_data)):
            group_size = sorted_group_data[i]
            people = group_data[group_size]
            num_people = 0
            for person in people:
                if person in people_to_assign:
                    num_people += 1
            if num_people > group_size:
                analyze_characteristics(people_map, people)
            elif num_people < group_size and not combined:
                combine_adjacent(people_map, group_data)
                combined = True
        if modifications == 0 and len(people_to_assign) > 2:
            tolerance += 1
            modifications = 1
        if len(people_to_assign) <= 2 and len(people_to_assign) > 0:
            people_to_add = []
            for i in range(len(people_to_assign)):
                people_to_add.append(people_to_assign[i])
            groups.append(people_to_add)
            people_to_assign = []
# remove people from the queue
# if too many people in group, go to next step of analysis
# if too little people in group, work on combining with adjacent people - same day wise

{'4-4-2021': {4: ['Dara Oseyemi', 'Connie Xu'], 3: ['Dara Oseyemi', 'Connie Xu']}, '4-5-2021': {4: ['Peter Bolgucki', 'Connie Xu', 'Chris Eisgruber'], 7: ['Sachiko Datta'], 3: ['Chris Eisgruber', 'Peter Bolgucki', 'Connie Xu']}, '4-6-2021': {4: ['Connie Xu', 'Sophie Li', 'Chris Eisgruber'], 3: ['Allen Park', 'Dara Oseyemi']}, '4-7-2021': {4: ['Connie Xu', 'Sophie Li', 'Chris Eisgruber'], 3: ['Allen Park'], 7: ['Sachiko Datta']}, '4-8-2021': {4: ['Sophie Li'], 3: ['Dara Oseyemi']}}
{'4-4-2021': {4: ['Dara Oseyemi'], 3: ['Dara Oseyemi']}, '4-5-2021': {4: ['Peter Bolgucki'], 7: ['Sachiko Datta'], 3: ['Peter Bolgucki']}, '4-6-2021': {4: [], 3: ['Allen Park', 'Dara Oseyemi']}, '4-7-2021': {4: [], 3: ['Allen Park'], 7: ['Sachiko Datta']}, '4-8-2021': {4: [], 3: ['Dara Oseyemi']}}


In [13]:
groups

[['Connie Xu', 'Sophie Li', 'Chris Eisgruber'],
 ['Allen Park', 'Dara Oseyemi'],
 ['Sachiko Datta', 'Peter Bolgucki']]

In [14]:
people_to_assign

[]

In [15]:
s1 = "{'4-4-2021': {4: ['Dara Oseyemi', 'Connie Xu'], 3: ['Dara Oseyemi', 'Connie Xu']}, '4-5-2021': {4: ['Peter Bolgucki', 'Connie Xu', 'Chris Eisgruber'], 7: ['Sachiko Datta'], 3: ['Chris Eisgruber', 'Peter Bolgucki', 'Connie Xu']}, '4-6-2021': {4: ['Connie Xu', 'Sophie Li', 'Chris Eisgruber'], 3: ['Allen Park', 'Dara Oseyemi']}, '4-7-2021': {4: ['Connie Xu', 'Sophie Li', 'Chris Eisgruber'], 3: ['Allen Park'], 7: ['Sachiko Datta']}, '4-8-2021': {4: ['Sophie Li'], 3: ['Dara Oseyemi']}}"
s1.replace("'", "\"")

'{"4-4-2021": {4: ["Dara Oseyemi", "Connie Xu"], 3: ["Dara Oseyemi", "Connie Xu"]}, "4-5-2021": {4: ["Peter Bolgucki", "Connie Xu", "Chris Eisgruber"], 7: ["Sachiko Datta"], 3: ["Chris Eisgruber", "Peter Bolgucki", "Connie Xu"]}, "4-6-2021": {4: ["Connie Xu", "Sophie Li", "Chris Eisgruber"], 3: ["Allen Park", "Dara Oseyemi"]}, "4-7-2021": {4: ["Connie Xu", "Sophie Li", "Chris Eisgruber"], 3: ["Allen Park"], 7: ["Sachiko Datta"]}, "4-8-2021": {4: ["Sophie Li"], 3: ["Dara Oseyemi"]}}'